[View in Colaboratory](https://colab.research.google.com/github/hamil168/Chatbots/blob/master/Seq2Seq.ipynb)

In [0]:

import numpy as np
import tensorflow as tf
import time


In [0]:

# Create placeholder for inputs and the targets
# in TF, all variables are tensors
# need to go from NP --> TF tensors
# need placeholders for every TF variables inputs and targets

def model_inputs():
  #inputs and targets are 2D matrices
  inputs = tf.placeholder(tf.int32, [None, None], name = 'input') 
  targets = tf.placeholder(tf.int32, [None, None], name = 'target')
  
  
  lr = tf.placeholder(tf.float32, name = 'learning_rate')
  keep_prob = tf.placeholder(tf.float32, name = 'keep_prop') #dropout
  
  return inputs, targets, lr, keep_prob

In [0]:
# preprocessing the targets
# need batches, 
# need each to start with <SOS> token

def preproc_targets(targets, word2int, batch_size):
  left_side = tf.fill([batch_size, 1], word2int['<SOS>'])
    
  # up to batch size less the last token
  # slide of [1,1]
  right_side = tf.strided_slice(targets, [0,0], [batch_size, -1], [1,1])
  preprocessed_target = tf.concat([left_side,right_side], axis=1)
  return preproc_targets